## WRF Analysis
### Created by Stanley Akor; Edited by Rachel Grena

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

## Helper Functions

In [2]:
'''
Converts latitude and longitude coordinates to WRF x and y indices.

Each 1km grid in a typical WRF output file is represented by an x and y index. 
To compare observation values in lat/lon to a WRF output, we first convert the lat/lon to WRF indices (i.e., x, y).

Parameters: 
    geog - File containing latitude and longitude coordinates in a typical WRF output file at 1km resolution.
    lat - Latitude of a site to be converted to the WRF x index.
    lon - Longitude of a site to be converted to the WRF y index.

Returns:
    ixlat - The x index of the site.
    iylon - The y index of the site.
'''

def get_wrf_xy(geog, lat, lon):
    # Example usage: lat = 38.89, lon = -106.95
    xlat = geog.XLAT.values[0, :, :]
    xlon = geog.XLONG.values[0, :, :]
    dist = np.sqrt((xlat - lat)**2 + (xlon - lon)**2)
    mindist = dist.min()
    ixlat = np.argwhere(dist == mindist)[0][0]
    iylon = np.argwhere(dist == mindist)[0][1]
    return ixlat, iylon


## Import File

In [ ]:
wrf_file = xr.open_dataset("WY_2023_wrf.nc") # , engine="netcdf4")

C:\Users\grena\anaconda3\envs\Mores_Creek\Lib\site-packages\xarray\backends\api.py:668: RuntimeWarning: 'scipy' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)


In [ ]:
'''
    The wrf_file contains 9 variables on an hourly time step for the 2023 water year. Of particular interest are:
    
        T2: Temperature at 2-metres (Celsitus)
        SNOW: Snow water equivalent (mm)
        PRCP: Deaccumulated precipitation (daily precipitation (rain) in mm)
        SNOWH: Snow height (m)
        ACPRCP: Accumulated precipitation from start of the water year (mm)

'''
wrf_file

In [ ]:
def get_wrf_data(file,lat,lon,variable):

    # open the wrf output file and extract the variable
    wrf_file = xr.open_dataset(file)

    # get wrf x/y indices for the site
    ixlat, iylon = get_wrf_xy(wrf_file, lat, lon)

    # collect data from the site

    site = wrf_file[variable].isel(south_north = ixlat, west_east = iylon)

    # create a dataframe for the site
    time =  site['XTIME'].values

    
    data = pd.DataFrame({'timestamp': time,
                         variable: site})

    # Convert the 'timestamp' column to datetime format
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Set 'timestamp' column as the index
    data.set_index('timestamp', inplace=True)

    return data

<span style="color: red;">Code Cells are kept raw to save time when generating results for specific variables.</span>

## Temperature

## Save results to CSV

## Temparture Alternative Method Analysis: Defining Fixed Window and Moving Average Controls

## SNOW

## Precipiation

In [ ]:
# Total Precip volume for all Sites
variable = 'PRCP'  # change as need: ("PRCP", 'ACPRCP', 'T2', 'SNOW')
path_to_wrf_file = "WY_2023_wrf.nc"

#define rc_variables
rc_variables = {
    'RC_01': {'lat': 43.64792811, 'lon': -115.9897596},
    'RC_02': {'lat': 43.74783308, 'lon': -115.8804603},
    'RC_03': {'lat': 43.7511991, 'lon': -115.9428652},
    'RC_04': {'lat': 43.79962177, 'lon': -115.7885124},
    'RC_05': {'lat': 43.90746942, 'lon': -115.6930072},
    'RC_06': {'lat': 43.92414139, 'lon': -115.6825098},
    'RC_07': {'lat': 43.9319943, 'lon': -115.66589},
    'RC_08': {'lat': 43.93639134, 'lon': -115.679588},
    'RC_09': {'lat': 43.94413863, 'lon': -115.6832378},
    'RC_10': {'lat': 43.95814402, 'lon': -115.6924836}
}

# Create an empty dictionary to store the results
results = {}

# Loop over each item in rc_variables
for rc, coords in rc_variables.items():
    lat = coords['lat']
    lon = coords['lon']
    
    # Get the data
    df = get_wrf_data(path_to_wrf_file, lat, lon, variable)
    
    # Resample from hourly to daily and aggregate using the sum
    df_precip = df.resample('D').sum()
    
    # Calculate the total sum and store it in the dictionary
    results[rc] = df_precip.sum()

# Print the results
for rc, total in results.items():
    print(f"{rc}: {total}")

### Cold and Warm Season Sums for Sinusoidal Curves 

In [ ]:
variable = 'PRCP'  # change as need: ("PRCP", 'ACPRCP', 'T2', 'SNOW')
path_to_wrf_file = "WY_2023_wrf.nc"
#Site_no 10: Crossing Point at x=152.20, y=-16.17 on 2023-06-01
#Site_no 10: Crossing Point at x=319.22, y=-16.14 on 2023-11-15
rc_variables = {
     'RC_10': {'lat': 43.95814402, 'lon': -115.6924836}
}

# Create an empty dictionary to store the results
results = {}

# Loop over each item in rc_variables
for rc1, coords in rc_variables.items():
    lat = coords['lat']
    lon = coords['lon']
    
    # Get the data
    df = get_wrf_data(path_to_wrf_file, lat, lon, variable)
    
    # Calculate the sum for each date range and store it in the dictionary
    results[rc1] = {
        '2022-10-05 to 2022-11-14': df['2022-10-05' : '2022-11-14'].sum(), # summer
        '2023-06-01 to 2023-11-03': df['2023-06-01' : '2023-11-03' ].sum(), # summer
        '2022-11-15 to 2023-05-31': df['2022-11-15' : '2023-05-31'].sum() #winter
    }

# Print the results
for rc1, sums in results.items():
    print(f"{rc1}: {sums}")

#### Precipitation Accumulation: Moving Average (7 day) per site
Each cell represents each rain collector (RC) and interval board (IB) site. 